In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [2]:
df = pd.read_csv('features.csv')

In [3]:
df.reset_index(drop = True,inplace = True)
df.drop(df.columns[0],axis=1 , inplace = True)
df

,product_id,seller_index,category_index,review_score
0,fd25ab760bfbba13c198fa3b4f1a0cd3,328,0,4.7
1,3880d25d502b15b1de6fddc42ad1d67a,1725,0,5.0
2,a76bb75d78ce6aa4a5202051b4f88a83,2208,0,3.8
3,ee92961b88199a0d4329a6beca8f63da,1791,0,4.3
4,adb1f1939b613f3078806b6afcca0e51,2648,0,4.7
...,...,...,...,...
32946,9c313adb4b38a55b092f53f83f78be9e,1266,71,3.0
32947,eacb104882d39ffb53140b1d1860a7c3,1266,71,3.0
32948,56a1efb30499b4dffd45c8d655199b11,63,71,5.0
32949,6c7a0a349ad11817745e3ad58abd5c79,2248,72,4.0


In [4]:
vector = df[['category_index','seller_index','review_score']].values
item_ids = df.index
product_id = df.product_id

In [5]:
def get_product(item_id):
    idx = item_ids[item_id]
    item_profile = vector[idx:idx+1]
    return item_profile
def get_related_products(ids):
    item_profile_list = [get_product(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profile_list)
    return item_profiles

In [6]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self,item_ids=None):
        self.item_ids = item_ids
    def get_model_name(self):
        return self.MODEL_NAME
    def _get_similar_products(self, item_id, topn=1000):
        cosine_similarities = cosine_similarity(get_product(item_id),vector)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_products = sorted([(item_ids[i],cosine_similarities[0,i]) for i in similar_indices ], key=lambda x : -x[1])
        return similar_products
    def get_recommendation(self,item_id,items_to_ignore = [], topn=10, verbose = False):
        items_to_ignore.append(item_id)
        similar_items = self._get_similar_products(item_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        return similar_items_filtered[:10]

In [7]:
csm = ContentBasedRecommender(item_ids)

In [8]:
csm = ContentBasedRecommender(item_ids)
x = csm.get_recommendation(5000)[:5]
ids = []
for i in range(0,len(x)):
    ids.append(x[i][0])
pids = []
for i in range (0,len(ids)):
    pids.append(product_id[ids[i]])
pids
product_id[5000]

'e923da464bbde92908a4d934bca4ab85'

In [9]:
DF = pd.read_csv('features_d.csv')
DF.drop(DF.columns[0],axis=1 , inplace = True)
DF[DF.product_id == pids[0]]

,product_id,review_score,product_name,product_category_name_english,product_weight_g,product_length_cm,product_height_cm,product_width_cm,price,seller_index,category_index
5043,5e9f7e47bbd1cf91d9a98ed9fdaf6193,5.0,garden_tools Product 1881,garden_tools,1050.0,40.0,6.0,22.0,105.2,2285,2


In [10]:
DF

,product_id,review_score,product_name,product_category_name_english,product_weight_g,product_length_cm,product_height_cm,product_width_cm,price,seller_index,category_index
0,fd25ab760bfbba13c198fa3b4f1a0cd3,4.7,sports_leisure Product 1536,sports_leisure,1300.0,30.0,30.0,35.0,185.0,328,0
1,3880d25d502b15b1de6fddc42ad1d67a,5.0,sports_leisure Product 2292,sports_leisure,9850.0,105.0,35.0,50.0,1199.0,1725,0
2,a76bb75d78ce6aa4a5202051b4f88a83,3.8,sports_leisure Product 506,sports_leisure,300.0,19.0,12.0,16.0,35.4,2208,0
3,ee92961b88199a0d4329a6beca8f63da,4.3,sports_leisure Product 1365,sports_leisure,450.0,35.0,7.0,25.0,39.9,1791,0
4,adb1f1939b613f3078806b6afcca0e51,4.7,sports_leisure Product 591,sports_leisure,4250.0,20.0,20.0,20.0,119.8,2648,0
...,...,...,...,...,...,...,...,...,...,...,...
32946,9c313adb4b38a55b092f53f83f78be9e,3.0,arts_and_craftmanship Product 2774,arts_and_craftmanship,500.0,16.0,6.0,16.0,19.0,1266,71
32947,eacb104882d39ffb53140b1d1860a7c3,3.0,arts_and_craftmanship Product 2778,arts_and_craftmanship,500.0,21.0,7.0,14.0,39.9,1266,71
32948,56a1efb30499b4dffd45c8d655199b11,5.0,arts_and_craftmanship Product 2770,arts_and_craftmanship,100.0,22.0,2.0,15.0,16.5,63,71
32949,6c7a0a349ad11817745e3ad58abd5c79,4.0,security_and_services Product 44,security_and_services,1225.0,27.0,35.0,15.0,183.3,2248,72
